In [25]:
import json
import datetime
import graphviz

In [26]:
# id mapping function
def clusterID_to_blueID(id):
    '''
    convert clusterIDs to IDs that display on the final visualization
    e.g. 0820_01 --> A01
    '''
    blueID = new_timepoint[id[:4]]+id[-2:]
    return blueID

def set_edge_style(edgeType, penweight = 3):
    '''
    set the edge style for different transition types
    '''
    if edgeType == 'unchanged':
        return ' [dir=none, weight=1, penwidth={}, color=azure3]'.format(penweight)
    elif edgeType == 'absorbed':
        return ' [dir=none, weight=1, penwidth={}, color=chartreuse3]'.format(penweight)
    elif edgeType == 'split':
        return ' [dir=none, weight=1, penwidth={}, color=crimson] '.format(penweight)
    elif edgeType == 'dissolved':
        return ' [dir=none, weight=1, penwidth={}, color=deepskyblue]'.format(penweight)
    elif edgeType == 'merged':
        return ' [dir=none, weight=1, penwidth={}, color=blueviolet]'.format(penweight)
    elif edgeType == 'reappear':
        return ' [dir=none, weight=1, penwidth={}, color=chocolate1, style=dashed]'.format(penweight)
    else: return ''
    
def find_max(value1, value2):
    '''
    helper function that find the max value
    '''
    if value1[4] > value2[4]:
        return value2
    else:
        return value1

def path_validation(filename):
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

def transition_viz(file_name):   
    '''
    main function to generate the visualization for cluster trnasitions
    '''
    # constants
    # change to your file path when use this note book
    input_path = './data/results/{}.json'.format(file_name) 
    output_path = './data/dot/{}'.format(file_name)
    dot_path = './data/dot/{}.dot'.format(file_name)

    transitions = list()
    sorted_nodeid = list()

    # initialize date range for [0819, 0902]
    date_range = list()
    start = datetime.datetime.strptime("19-08-2020", "%d-%m-%Y")
    end = datetime.datetime.strptime("03-09-2020", "%d-%m-%Y")
    date_range_dt = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    for dt in date_range_dt:
        str_date = str(dt.month).zfill(2) + str(dt.day).zfill(2) # pad casted string to 2 digit with leading 0s
        date_range.append(str_date)

    # mapping date to letters
    new_timepoint = dict(zip(date_range,string.ascii_uppercase))

    # read json file and load the transition metadata
    with open(input_path, 'r', encoding='utf-8') as textfile: 
        transitions = json.load(textfile)

    copy_trans = transitions
    for i, value1 in enumerate(copy_trans):
        for j, value2 in enumerate(copy_trans):
            if value1[0] == value2[0] and value1[1] == value2[1] and value1[2] == value2[2]:
                if value1[3] == 'weak' and value2[3] == 'medium':
                    transitions.remove(find_max(value1, value2))
                elif value1[3] == 'medium' and value2[3] == 'weak':
                    transitions.remove(find_max(value1, value2))
                elif value1[3] == 'strong' and value2[3] == 'medium':
                    transitions.remove(find_max(value1, value2))
                elif value1[3] == 'medium' and value2[3] == 'strong':
                    transitions.remove(find_max(value1, value2))

    for i in transitions:
        if i[0] not in sorted_nodeid:
            sorted_nodeid.append(i[0])
        if i[1] not in sorted_nodeid:
            sorted_nodeid.append(i[1])
    sorted_nodeid = sorted(sorted_nodeid)  

    path_validation(dot_path)
    # create the dot file
    with open(dot_path, 'w') as f:
        f.write('digraph A {\nranksep=.75; nodesep = 0.15; rankdir = LR;\ngraph [fontname=Verdana ];\nnode [fontname=Verdana];\nedge [fontname=Verdana];{\nnode [shape=plaintext, fontname=Verdana, fontsize=18];\n')
        for i,date in enumerate(date_range):
            if i > 0:
                f.write(' -> ')
            f.write(date)
        f.write(';\n}\n')

        f.write('{ rank = same; ')
        for i,nodeid in enumerate(sorted_nodeid):
            if date != nodeid[:4] and i != 0:
                f.write('}\n')
                f.write('{ rank = same; ')
            date = nodeid[:4]
            f.write(clusterID_to_blueID(nodeid) + '; ')
        f.write('}\n')

        for i in transitions:
            m = 1
            if i[3] == 'weak':
                m = 0.1
            elif i[3] == 'medium':
                m = 0.8
            elif i[3] == 'strong':
                m = 1
            f.write(clusterID_to_blueID(i[0]) + '->' + clusterID_to_blueID(i[1]) + set_edge_style(i[2],7*i[4]*m) + ';\n')
        f.write('}\n')

    # save the figure as a png file
    src = graphviz.Source.from_file(dot_path)
    src.format  = 'png'
    src.render(output_path, view=True)

In [27]:
# example
transition_viz('fuzzy_graph_tuples')